In [1]:
import pandas as pd
import requests
import json
import os
from pathlib import Path

In [2]:
def prep_data(url):
    '''Preps metro city state code files from US Census site.'''
    df = pd.read_excel(url)
    #Colnames in this row
    colnames = df.iloc[1, :].values
    df = df.iloc[2:-4, :]
    df.columns = colnames
    df = df.reset_index().drop('index', axis = 1)
    return df

In [3]:
#Reading in urls to grab relevant data.
with open('metro_city_states.json') as f:
    urls = json.load(f)

In [4]:
dfs = {'ms_df': prep_data(urls['ms_url']), 'mc_df': prep_data(urls['mc_url'])}
home = Path.cwd()
os.chdir(Path(home).parents[0])
#Geocoded prepped data.
df = pd.read_csv('10_11_20_geocoded.csv')
os.chdir(home)

C:\Users\Chris_D\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
cb = 'CBSA Code'
c = 'Metropolitan/Micropolitan Statistical Area'
val = 'Metropolitan Statistical Area'

#Ensuring CBSA codes are strings.
#Keeping only metros for now to get rid of noise when considering micros
#Micros result in more than one record per city. Ex CBSA code: 10100
for k, v in dfs.items():
    v.loc[:, cb] = v.loc[:, cb].astype(str)
    dfs[k] = v[v.loc[:, c] == val]
    
#Verified only Metro.
# dfs['ms_df'][c].unique()
# dfs['mc_df'][c].unique()

In [6]:
#Only keeping columns needed and dropping duplicates.
cols = [cb, 'CBSA Title', 'FIPS State Code', 'Principal City Name']
dfs['mc_df'] = dfs['mc_df'][cols].drop_duplicates()
cols2 = cols[:-1] + ['State Name', 'Metropolitan Division Code', 'Metropolitan Division Title', 'CSA Code', 'CSA Title']
dfs['ms_df'] = dfs['ms_df'][cols2].drop_duplicates()

In [7]:
len(dfs['mc_df']) #702
len(dfs['ms_df']) #465

465

In [8]:
#Since filtered to Metropolitan above don't need as key for join.
#Some have more than one. Will analyze further.
merged = pd.merge(dfs['ms_df'], dfs['mc_df'], on = list(dfs['ms_df'].columns[0:3])) #838

In [9]:
#Obtaining dict of State names and State abbreviations.
#https://gist.github.com/rogerallen/1583593
git = urls['gt_url']
git_txt = requests.get(git).text
beg_ind = git_txt.index('{')
end_ind = git_txt.index('}')
state_dict = eval(' '.join(git_txt[beg_ind: end_ind + 1].split()))
abbrev_us_state = dict(map(reversed, state_dict.items()))

In [10]:
#Mapping dict to new series in df.
df.loc[:, 'State_Name'] = df.State_Province.map(abbrev_us_state)

In [11]:
#Reads like the same amount of cities, which should be correct. Some cities associated (like "tri-state" areas).
# merged[['State Name', 'Principal City Name', 'CBSA Code']].groupby(['State Name', 'Principal City Name']).count()
len(df)

df = pd.merge(df, merged[['CBSA Code', 'Principal City Name', 'State Name']].drop_duplicates(),
              left_on = ['City', 'State_Name'], right_on = ['Principal City Name', 'State Name'],
              how = 'left')
df = df.drop('State Name', axis = 1)

#Some have no result CBSA codes after merge, but after checking a few, it's likely because NOT in Metros.
#4689 without CBSA code.
# df[df.loc[:, 'CBSA Code'].isnull()]

In [22]:
#95 unique metros, those with no nan values or are not based in micros.
distinct_cbsas = [code for code in df.loc[:, 'CBSA Code'].unique() if str(code) != 'nan']

In [24]:
len(df) #25607
#20918 records left.
#df[df.loc[:, 'CBSA Code'].isin(distinct_cbsas)]

25607

In [29]:
# df.to_csv(Path(home).parents[0].joinpath('10_11_20_geoc_metros.csv'), index = False)